In [27]:
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.sampledata.periodic_table import elements
from bokeh.transform import dodge, factor_cmap

output_notebook()

Loading BokehJS ...

## Perpare PSE

In [49]:
periods = ["I", "II", "III", "IV", "V", "VI", "VII"]
groups = [str(x) for x in range(1, 19)]

df = elements.copy()
df["atomic mass"] = df["atomic mass"].astype(str)
df["group"] = df["group"].astype(str)
df["period"] = [periods[x - 1] for x in df.period]
df = df[df.group != "-"]
df = df[df.symbol != "Lr"]
df = df[df.symbol != "Lu"]

## Read our parsing statistics 

In [50]:
df_statistics = pd.read_pickle("statistics.pkl")

## Merge them

In [51]:
df = df.merge(df_statistics, left_on="symbol", right_on="element", how="left")
df.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,counter_x,element,ox_states_x,counter_y,ox_states_y,count_x,count_y,mof_normalized,all_normalized,difference
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,"{1: 0.9849624060150376, 2: 0.00751879699248120...",Li,"[1, 2, 4]","{1: 0.9941176470588236, 2: 0.0058823529411764705}","[1, 2]",133.0,170.0,0.002415,0.001859,0.000556
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,{2: 1.0},Be,[2],{2: 1.0},[2],2.0,16.0,0.000036,0.000175,0.000139
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Add some abundance in the earth's crust

In [20]:
df_abundance = pd.read_csv("../oximachine_featurizer/data/abundance_data.csv")

In [54]:
df = df.merge(df_abundance, left_on="symbol", right_on="Sy", how="left")
df.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,Boil [K],Heat [J(g*K)],EN,Abundance [mg/kg],Single Bond [pm] (Cordero),Single Bond [pm] (Pyykkö),Double Bond [pm] (Pyykkö),Triple Bond [pm] (Pyykkö),vdW Radius [pm],Electron configuration
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,20.28,14.304,2.2,1400.000,31,32,–,–,120,1s<sup>1</sup>
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,4.22,5.193,–,0.008,28,46,–,–,140,1s<sup>2</sup>
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,1560,3.582,0.98,20.000,128,133,124,–,182,[He] 2s<sup>1</sup>
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,2742,1.825,1.57,2.800,96,102,90,85,153,[He] 2s<sup>2</sup>
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,4200,1.026,2.04,10.000,84,85,78,73,192,[He] 2s<sup>2</sup> 2p<sup>1</sup>


In [55]:
df["rel_abundance"] = df["Abundance [mg/kg]"] / df["Abundance [mg/kg]"].sum()

In [56]:
df.head()

,atomic number,symbol,name,atomic mass,CPK,electronic configuration,electronegativity,atomic radius,ion radius,van der Waals radius,...,Heat [J(g*K)],EN,Abundance [mg/kg],Single Bond [pm] (Cordero),Single Bond [pm] (Pyykkö),Double Bond [pm] (Pyykkö),Triple Bond [pm] (Pyykkö),vdW Radius [pm],Electron configuration,rel_abundance
0,1,H,Hydrogen,1.00794,#FFFFFF,1s1,2.20,37.0,NaN,120.0,...,14.304,2.2,1400.000,31,32,–,–,120,1s<sup>1</sup>,1.395917e-03
1,2,He,Helium,4.002602,#D9FFFF,1s2,NaN,32.0,NaN,140.0,...,5.193,–,0.008,28,46,–,–,140,1s<sup>2</sup>,7.976668e-09
2,3,Li,Lithium,6.941,#CC80FF,[He] 2s1,0.98,134.0,76 (+1),182.0,...,3.582,0.98,20.000,128,133,124,–,182,[He] 2s<sup>1</sup>,1.994167e-05
3,4,Be,Beryllium,9.012182,#C2FF00,[He] 2s2,1.57,90.0,45 (+2),NaN,...,1.825,1.57,2.800,96,102,90,85,153,[He] 2s<sup>2</sup>,2.791834e-06
4,5,B,Boron,10.811,#FFB5B5,[He] 2s2 2p1,2.04,82.0,27 (+3),NaN,...,1.026,2.04,10.000,84,85,78,73,192,[He] 2s<sup>2</sup> 2p<sup>1</sup>,9.970836e-06


## Classic periodic table

In [62]:
cmap = {
    "alkali metal": "#a6cee3",
    "alkaline earth metal": "#1f78b4",
    "metal": "#d93b43",
    "halogen": "#999d9a",
    "metalloid": "#e08d49",
    "noble gas": "#eaeaea",
    "nonmetal": "#f1d4Af",
    "transition metal": "#599d7A",
}

TOOLTIPS = [
    ("Name", "@name"),
    ("Atomic number", "@{atomic number}"),
    ("Atomic mass", "@{atomic mass}"),
    ("Type", "@metal"),
    # ("Electronic configuration", "@{electronic configuration}"),
]

p = figure(
    title="Periodic Table",
    plot_width=1000,
    plot_height=450,
    x_range=groups,
    y_range=list(reversed(periods)),
    tools="hover",
    toolbar_location=None,
    tooltips=TOOLTIPS,
)

r = p.rect(
    "group",
    "period",
    0.95,
    0.95,
    source=df,
    fill_alpha=0.6,
    legend="metal",
    color=factor_cmap("metal", palette=list(cmap.values()), factors=list(cmap.keys())),
)

text_props = {"source": df, "text_align": "left", "text_baseline": "middle"}

x = dodge("group", -0.4, range=p.x_range)

p.text(x=x, y="period", text="symbol", text_font_style="bold", **text_props)

p.text(
    x=x,
    y=dodge("period", 0.3, range=p.y_range),
    text="atomic number",
    text_font_size="8pt",
    **text_props,
)

p.text(
    x=x,
    y=dodge("period", -0.35, range=p.y_range),
    text="name",
    text_font_size="5pt",
    **text_props,
)

p.text(
    x=x,
    y=dodge("period", -0.2, range=p.y_range),
    text="atomic mass",
    text_font_size="5pt",
    **text_props,
)

p.text(
    x=["3", "3"],
    y=["VI", "VII"],
    text=["LA", "AC"],
    text_align="center",
    text_baseline="middle",
)

p.outline_line_color = None
p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_standoff = 0
p.legend.orientation = "horizontal"
p.legend.location = "top_center"
p.hover.renderers = [r]  # only hover element boxes


show(p)